In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

from loguru import logger

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util
util.startLoop()

# Imports 
from utils import Vars, get_pickle

# Set the root
from from_root import from_root
ROOT = from_root()

# Generate `margin` scale for the nse symbol

### Get an option chain from nse web

In [ ]:
from utils import nse_web_json_to_df, get_lots_from_nse, clean_nse_web_options, get_dte, this_is_nse_index

In [ ]:
# Input
symbol = 'TCS'

In [ ]:
def get_an_nse_option_chain(symbol: str) -> pd.DataFrame:

    df = nse_web_json_to_df(symbol)\
            .pipe(clean_nse_web_options)

    # get the dte
    dte = df.expiry.apply(lambda x: get_dte(x, exchange='NSE'))
    df.insert(5, 'dte', dte)

    return df

In [ ]:
# nse_web_json_to_df(symbol)
df_opts = get_an_nse_option_chain(symbol=symbol)

In [ ]:
df_opts.head()

## Get `lot size` from SAMCO

### Build data dictionary for `lot_size` extraction

In [ ]:
import requests
from utils import this_is_nse_index
import json
from tqdm import tqdm

In [ ]:
def get_data_from_samco(url: str, data: dict) -> requests.models.Response:

    """Gets data from SAMCO"""

    base_url = 'https://www.samco.in'

    samco_headers = {
        'User-Agent': 'python-requests/2.31.0', 
        'Accept-Encoding': 'gzip, deflate', 
        'Accept': '*/*', 
        'Connection': 'keep-alive', 
        "Referer": "https://www.samco.in/calculators/span-margin-calculator"}

    with requests.Session() as session:
        request = session.get(base_url, headers=samco_headers, timeout=5)
        cookies = dict(request.cookies)
        response = session.post(url, data=data, headers=samco_headers, cookies=cookies, timeout=5)

    return json.loads(response.text)
    # return response
    

In [ ]:
def get_qty_from_samco(df_opts: pd.DataFrame) -> pd.DataFrame:

        """Gets lot size from SAMCO"""
        df = df_opts.copy()
        
        url = 'https://www.samco.in/span/get_quntity_datewise'

        # Build quantity dict
        symbol = df.symbol.iloc[0]
        expiry_style = "weekly" if this_is_nse_index(symbol) else "monthly"

        if this_is_nse_index(symbol):
                df.insert(4, 'exp', df.expiry.dt.strftime('%d%b%y').str.upper())
        else:
                df.insert(4, 'exp', df.expiry.dt.strftime('%d%b').str.upper())

        an_expiry = df.exp.iloc[0]

        data = {'name': symbol,
                'expiry': an_expiry,
                'exchange': 'NFO',
                'product': 'OPT',
                'table': expiry_style,
        }

        qty = get_data_from_samco(url, data)
        df = df.assign(qty=qty, quantity=qty)

        return df

In [ ]:
df = get_qty_from_samco(df_opts)
df.head()

# Get all margins from SAMCO

### Build data dictionary for margins

In [ ]:
def data_dict_for_samco_margins(df: pd.DataFrame) -> dict:

    # map df's columns to SAMCOs
    cols_dict = {
        'symbol': 'underlying',
        'strike': 'strike_price',
        'exp': 'expiry',
        'qty': 'qty',
        'quantity': 'quantity',
    }

    # filter out only required dict columns for margins
    df_dict = df.drop(columns='expiry')\
            .rename(columns=cols_dict)[cols_dict.values()]


    df_dict = df_dict.assign(quantity = df_dict.qty, 
            exchange = "NFO",
            product = "OPT",
            option = "PE",
            buy_sell = "sell")

    data_dict = dict(zip(df_dict.index, df_dict.to_dict(orient='records')))

    return data_dict


In [ ]:
data_dicts = data_dict_for_samco_margins(df)

## Async function for SAMCO with a limit on simultaneous tasks

### Note: Did'nt work when SAMCO site was not generating margins in morning

In [ ]:
# !!! Didn't work as site was giving 0 for all margins

import aiohttp
import asyncio
import json
from tqdm import tqdm

async def coro_samco_margin(url: str, data: dict) -> dict:
    """
    [async] Get margins from SAMCO for data dictionary with {index, data} construct
    """
    base_url = 'https://www.samco.in'

    headers = {
        'User-Agent': 'python-requests/2.31.0',
        'Accept-Encoding': 'gzip, deflate',
        'Accept': '*/*',
        'Connection': 'keep-alive',
        "Referer": "https://www.samco.in/calculators/span-margin-calculator"
    }

    async with aiohttp.ClientSession() as session:
        async with session.get(base_url, headers=headers, timeout=5) as request:
            cookies = dict(request.cookies)
            async with session.post(url, data=data, headers=headers, cookies=cookies, timeout=5) as response:
                m_dict = json.loads(await response.text())

    return m_dict

async def get_samco_margins(data_dict: dict, blk:int=15):
    url = 'https://www.samco.in/span/get_span_value'

    tasks = []
    margins = {}

    # Create tasks for each data
    for index, data in tqdm(data_dict.items(), desc='Getting margins'):
        task = asyncio.create_task(coro_samco_margin(url, data))
        tasks.append(task)

        # Process blk of tasks at a time
        if len(tasks) == blk:
            results = await asyncio.gather(*tasks)
            for i, result in enumerate(results):
                margins[index - len(tasks) + i + 1] = result.get('margin')

            tasks.clear()

    # Process remaining tasks
    if tasks:
        results = await asyncio.gather(*tasks)
        for i, result in enumerate(results):
            margins[len(data_dict) - len(tasks) + i + 1] = result.get('margin')

    return margins

In [ ]:
margins = asyncio.run(get_samco_margins(data_dicts, 5))

In [ ]:
margins

# Get strike boundaries

In [ ]:
# quantity = qty
# dfm = dfm.assign(qty=qty, quantity=quantity)

In [ ]:
def get_strike_boundaries_per_expiry(df_opts: pd.DataFrame) -> pd.DataFrame:
    """Highest and Lowest PE strikes per expiry. Needs cleaned nse web opts with `dte`"""

    df_pe = df_opts[df_opts['right'] == 'PE']
    grouped = df_pe.groupby('expiry')['strike']
    min_strike_rows = df_pe[df_pe['strike'] == grouped.transform('min')]
    max_strike_rows = df_pe[df_pe['strike'] == grouped.transform('max')]

    dfm = pd.concat([min_strike_rows, max_strike_rows]).sort_values('dte')
    dfm = get_qty_from_samco(dfm)

    # dfm.insert(4, 'exp', dfm.expiry.dt.strftime('%d%b%y').str.upper())
    # sym = set(dfm.symbol.to_numpy()).pop()
    # if this_is_nse_index(sym):
    #     dfm.insert(4, 'exp', dfm.expiry.dt.strftime('%d%b%y').str.upper())
    # else:
    #     dfm.insert(4, 'exp', dfm.expiry.dt.strftime('%d%b').str.upper())

    return dfm
    

In [ ]:
# extract puts only
df_puts = df_opts[df_opts.right == 'PE'].reset_index(drop=True)
dfm = get_strike_boundaries_per_expiry(df_puts)

In [ ]:
dfm.head()

In [ ]:
data_dict = data_dict_for_samco_margins(dfm)
margins = asyncio.run(get_samco_margins(data_dict))
margins

In [ ]:
url = 'https://www.samco.in/span/get_span_value'

margins = dict()
for k, data in tqdm(data_dict.items(), desc='Getting margins'):
    margins[k] = get_data_from_samco(url, data).get('margin')


In [ ]:
margins

In [ ]:
mgnCost = [index.get('total_margin') for index in dfm.index.map(margins)]
dfm = dfm.assign(mgnCost=mgnCost)

In [ ]:
g_puts = df_puts.groupby('dte')
g_m = dfm.groupby('dte')

### Experimenting with dfm to see the rom of ask_price, if available

In [ ]:
m1 = dfm.ask_price>0
m2 = dfm.right=='PE'
m3 = dfm.strike < dfm.undPrice
df = dfm[m1 & m2 & m3]
df.assign(rom=df.ask_price*df.qty/df.mgnCost*365/df.dte)

## Basic fishing from SAMCO

In [ ]:
import requests
import json

In [ ]:
# variable
underlying = "AARTIIND"
expiry = "30MAY"
strike_price = 720

# calculated
qty = 1000
quantity = qty


# fixed:
exchange = "NFO"
product = "OPT"
option = "PE"
buy_sell = "sell"

# build dictionary
data = {
    "underlying": underlying,
    "expiry": expiry,
    "strike_price": strike_price,
    "qty": qty,
    "quantity": quantity,
    "exchange": exchange,
    "product": product,
    "option": option,
    "buy_sell": buy_sell
}

In [ ]:
def get_samco_margin(data: dict) -> dict:

    """Gets margins from SAMCO
    
    `data` is a dict with:
    
    ### variable
    - underlying = "AARTIIND"
    - expiry = "30MAY"
    - strike_price = 720
    - qty = 1000    # should be equal to lotsize

    ### calculated

    - quantity = qty


    ### fixed:
    - exchange = "NFO"
    - product = "OPT"
    - option = "PE"
    - buy_sell = "sell" 
    
    """


    base_url = 'https://www.samco.in'
    url = 'https://www.samco.in/span/get_span_value'

    headers = {
        'User-Agent': 'python-requests/2.31.0', 
        'Accept-Encoding': 'gzip, deflate', 
        'Accept': '*/*', 
        'Connection': 'keep-alive', 
        "Referer": "https://www.samco.in/calculators/span-margin-calculator"}

    with requests.Session() as session:
        request = session.get(base_url, headers=headers, timeout=5)
        cookies = dict(request.cookies)
        response = session.post(url, data=data, headers=headers, cookies=cookies, timeout=5)
        
    m_dict = json.loads(response.text)

    mgn_dict = m_dict.get('margin')

    margins = \
        {'mgnCompute': mgn_dict.get('total_margin'),
        'required_mgn': mgn_dict.get('totalrequirement'),
        'exposure': mgn_dict.get('exposuremargin'),
        'span': mgn_dict.get('spanRequirement'),
        'addn_margin': mgn_dict.get('additionalmargin')
        }

    return margins

In [ ]:
get_samco_margin(data)

In [ ]:

for k, v in data_dict.items():
    margin_dict[k] = get_samco_margin(v)

In [ ]:
margin_dict

In [ ]:
data_dict.get(372)

In [ ]:
import grequests

In [ ]:
import aiohttp
import asyncio
import json

async def async_samco_margins(data: dict) -> dict:
    base_url = 'https://www.samco.in'
    url = 'https://www.samco.in/span/get_span_value'

    headers = {
        'User-Agent': 'python-requests/2.31.0',
        'Accept-Encoding': 'gzip, deflate',
        'Accept': '*/*',
        'Connection': 'keep-alive',
        "Referer": "https://www.samco.in/calculators/span-margin-calculator"
    }

    async with aiohttp.ClientSession() as session:
        async with session.get(base_url, headers=headers, timeout=5) as request:
            cookies = dict(request.cookies)
            async with session.post(url, data=data, headers=headers, cookies=cookies, timeout=5) as response:
                m_dict = json.loads(await response.text())
    
    return m_dict


In [ ]:
data = {
    'underlying': 'AARTIIND',
    'expiry': '30MAY',
    'strike_price': 720,
    'qty': 1000,
    'exchange': 'NFO',
    'product': 'OPT',
    'option': 'PE',
    'buy_sell': 'sell'
}

m_dict = asyncio.run(async_samco_margins(data=data))

mgn_dict = m_dict.get('margin')

margins = {
    'mgnCompute': mgn_dict.get('total_margin'),
    'required_mgn': mgn_dict.get('totalrequirement'),
    'exposure': mgn_dict.get('exposuremargin'),
    'span': mgn_dict.get('spanRequirement'),
    'addn_margin': mgn_dict.get('additionalmargin')
}

In [ ]:
margins